In [25]:
def convertToPolygonList(pointlist):
    points = []
    index = 0;
    
    x_val = 0
    for coordinate in pointlist.split(" "):
        if index == 0:
            x_val = int(float(coordinate))
        else:
            y_val = int(float(coordinate))
            points.append((x_val, y_val))
        index = (index + 1)  %2
    if index != 0:
        print("There must me as many x as y-coordinates.")
        return
    return points

In [26]:
def import_lon_lat_file(path):
    polygon_families = []
    current_family = []
    fobj = open(path)
    current_reading_type = 0;
    count_inners = 0;
    current_inners = []
    parsed = 0
    
    for line in fobj:
        if current_reading_type == 0:
            current_outer = convertToPolygonList(line);
            current_reading_type = 1
            current_family.append(current_outer)
        elif current_reading_type == 1:
            count_inners = int(line)
            if count_inners > 0:
                current_reading_type = 2
                parsed = 0
            else:
                current_reading_type = 0
                polygon_families.append(current_family)
                current_family = []
        else:
            cur_inner = convertToPolygonList(line)
            current_inners.append(cur_inner)
            parsed = parsed +1
            if parsed >= count_inners:
                current_reading_type =0
                current_family.extend(current_inners)
                polygon_families.append(current_family)
                current_family = []
                current_inners = []
                current_reading_type = 0
        
        
    fobj.close()
    return polygon_families

In [27]:
fileformat_plain = ".txt"
folder = "lon_lat_FMI_format/"
name = "brandenburg"
currentFile = folder + name 

polygon_families = import_lon_lat_file(currentFile + fileformat_plain)

In [106]:
from math import sqrt
import time

try:
    # Python3
    from queue import PriorityQueue
    from math import inf
except ImportError:
    # Python2
    from Queue import PriorityQueue
    inf = float("inf")


def _point_to_polygon_distance(x, y, polygon):
        
    #print "_point_to_polygon_distance"
    #print x
    #print y
    
    inside = False
    min_dist_sq = inf

    for ring in polygon:
        b = ring[-1]
        for a in ring:
           #print "first"
            #print a
            #print "last"
            #print b
            if ((a[1] > y) != (b[1] > y) and
                    (x < (b[0] - a[0]) * (y - a[1]) / (b[1] - a[1]) + a[0])):
                inside = not inside

            min_dist_sq = min(min_dist_sq, _get_seg_dist_sq(x, y, a, b))
            #print "min distatnce sq"
            #print min_dist_sq
            b = a

    result = sqrt(min_dist_sq)
    #print "--------------------------------------------------------"
   # print "inside:"
    #print inside
    
    #raise("bla")
    
    if not inside:
        return -result
    return result


def _get_seg_dist_sq(px, py, a, b):
   # print "px"
   # print px
   # print "py"
    #print py
    #print "a"
    #print a
    #print "b"
    #print b
    
    x = a[0]
    y = a[1]
    dx = b[0] - x
    dy = b[1] - y

    if dx != 0 or dy != 0:
        t = ((px - x) * dx + (py - y) * dy) / (dx * dx + dy * dy)

        if t > 1:
            x = b[0]
            y = b[1]

        elif t > 0:
            x += dx * t
            y += dy * t

    dx = px - x
    dy = py - y
    #print "result"
    result = dx * dx + dy * dy
    #print result 
    return result


class Cell(object):
    def __init__(self, x, y, h, polygon):
        self.h = h
        self.y = y
        self.x = x
        self.d = _point_to_polygon_distance(x, y, polygon)
        self.max = self.d + self.h * sqrt(2)


def _get_centroid_cell(polygon):
    area = 0
    x = 0
    y = 0
    points = polygon[0]
    b = points[-1]  # prev
    for a in points:
        f = a[0] * b[1] - b[0] * a[1]
        x += (a[0] + b[0]) * f
        y += (a[1] + b[1]) * f
        area += f * 3
        b = a
    if area == 0:
        return Cell(points[0][0], points[0][1], 0, polygon)
    return Cell(x / area, y / area, 0, polygon)

    pass


def polylabel(polygon, precision=1.0, debug=False):
    # find bounding box
    first_item = polygon[0][0]
    min_x = first_item[0]
    min_y = first_item[1]
    max_x = first_item[0]
    max_y = first_item[1]
    for p in polygon[0]:
        if p[0] < min_x:
            min_x = p[0]
        if p[1] < min_y:
            min_y = p[1]
        if p[0] > max_x:
            max_x = p[0]
        if p[1] > max_y:
            max_y = p[1]

  
    
    width = max_x - min_x
    height = max_y - min_y
    cell_size = min(width, height)
    h = cell_size / 2.0
 
    print type(h)
    print type(cell_size)
    #print "width"
    #print(width)
    #print "height"
    ##print(height)
    #print "cell size"
    #print(cell_size)
    #print "h"
    #print(h)
    
    #print("\nmax x_")
    #print(max_x)
    #print("\nminx")
    #print(min_x)
    
        
    cell_queue = PriorityQueue()

    if cell_size == 0:
        return [min_x, min_y]
    
    print (min_x, min_y, max_x, max_y, cell_size, int(cell_size))
    

    # cover polygon with initial cells
    for x in range(min_x, max_x, int(cell_size)):
        for y in range(min_y, max_y, int(cell_size)):
            print str(x) + ", " +  str(y)
            c = Cell(x + h, y + h, h, polygon)
            cell_queue.put((-c.max, time.time(), c))

    best_cell = _get_centroid_cell(polygon)
    #print("\ncentroid")

    bbox_cell = Cell(min_x + width / 2, min_y + height / 2, 0, polygon)
    #print("\nbbox")
    #print("\parameter")
    #print(min_x + width / 2)
    #print(min_y + height / 2)
    #print(bbox_cell.max)

    
    if bbox_cell.d > best_cell.d:
        best_cell = bbox_cell

    print('best_cell.max:')
    print(best_cell.max)
    print('best_cell.d:')
    print(best_cell.d)
    
    print("Los!")
    
    num_of_probes = cell_queue.qsize()
    while not cell_queue.empty():
        print "QSize" + str(cell_queue.qsize())
        _, __, cell = cell_queue.get()
        print "cell.d " + str(cell.d)+ "cell.maX " + str(cell.max)+ "cell.x " + str(cell.x)+ "cell.y " + str(cell.y)
        
        if cell.d > best_cell.d:
            best_cell = cell
            print('found best {}'.format(cell.d))

        if cell.max - best_cell.d <= precision:
            continue

            
        h = cell.h / 2
        print("split!, erg = " + str(cell.max - best_cell.d) + "new H" + str(h));
        c = Cell(cell.x - h, cell.y - h, h, polygon)
        cell_queue.put((-c.max, time.time(), c))
        print(c.y)
        c = Cell(cell.x + h, cell.y - h, h, polygon)
        print(c.y)
        cell_queue.put((-c.max, time.time(), c))
        c = Cell(cell.x - h, cell.y + h, h, polygon)
        print(c.y)
        cell_queue.put((-c.max, time.time(), c))
        c = Cell(cell.x + h, cell.y + h, h, polygon)
        print(c.y)
        cell_queue.put((-c.max, time.time(), c))
        num_of_probes += 4

    if debug:
        print('num probes: {}'.format(num_of_probes))
        print('best distance: {}'.format(best_cell.d))

    return [best_cell.x, best_cell.y]

In [107]:

#testpolygonfamiliy = [[[0,0],[10,0],[10,4],[12,6],[6,9],[0,0]],[[3,1],[5,1],[5,3],[3,3],[3,1]], [[5.5,5.5], [7.5,7.5], [5.5,7.5], [5.5,5.5]] ]
for polygonfamily in polygon_families:
    print polylabel(polygonfamily, 0.1)  # [0, 0]

<type 'float'>
<type 'int'>
(112838045, 513639044, 147658159, 535551190, 21912146, 21912146)
112838045, 513639044
134750191, 513639044
best_cell.max:
870151.167728
best_cell.d:
870151.167728
Los!
QSize2
cell.d 543754.321584cell.maX 16037981.3485cell.x 123794118.0cell.y 524595117.0
split!, erg = 15167830.1808new H5478036.5
519117080.5
519117080.5
530073153.5
530073153.5
QSize5
cell.d 119418.913715cell.maX 15613645.9407cell.x 145706264.0cell.y 524595117.0
split!, erg = 14743494.7729new H5478036.5
519117080.5
519117080.5
530073153.5
530073153.5
QSize8
cell.d 3870253.95728cell.maX 11617367.4708cell.x 140228227.5cell.y 519117080.5
found best 3870253.95728
split!, erg = 7747113.51347new H2739018.25
516378062.25
516378062.25
521856098.75
521856098.75
QSize11
cell.d 1651209.81532cell.maX 9398323.3288cell.x 129272154.5cell.y 530073153.5
split!, erg = 5528069.37152new H2739018.25
527334135.25
527334135.25
532812171.75
532812171.75
QSize14
cell.d 1562643.16932cell.maX 9309756.68279cell.x 14022822

520612974.989
520614312.4
520614312.4
QSize164
cell.d 5164815.26092cell.maX 5179946.342cell.x 141201862.894cell.y 520604281.815
split!, erg = 1817.50148713new H5349.64501953
520598932.17
520598932.17
520609631.46
520609631.46
QSize167
cell.d 5178023.96657cell.maX 5179915.35171cell.x 141232623.352cell.y 520613643.694
split!, erg = 1786.51119048new H668.705627441
520612974.989
520612974.989
520614312.4
520614312.4
QSize170
cell.d 5178770.68082cell.maX 5179716.37338cell.x 141230617.236cell.y 520614312.4
found best 5178770.68082
split!, erg = 945.692567563new H334.352813721
520613978.047
520613978.047
520614646.753
520614646.753
QSize173
cell.d 5179076.27111cell.maX 5179549.11739cell.x 141230951.588cell.y 520614646.753
found best 5179076.27111
split!, erg = 472.846283781new H167.17640686
520614479.576
520614479.576
520614813.929
520614813.929
QSize176
cell.d 5177555.79178cell.maX 5179447.17692cell.x 141229948.53cell.y 520616318.517
split!, erg = 370.905809107new H668.705627441
520615649.81

cell.d 5114722.82799cell.maX 5144984.99016cell.x 141319555.084cell.y 520593582.525
QSize104
cell.d 5017873.42713cell.maX 5138922.07578cell.x 141340953.664cell.y 520743372.586
QSize103
cell.d 3199795.17413cell.maX 5136573.5525cell.x 127902645.375cell.y 523225607.875
QSize102
cell.d 4646495.57452cell.maX 5130690.16912cell.x 141255359.344cell.y 521513721.469
QSize101
cell.d 4888407.51165cell.maX 5130504.80894cell.x 141084170.703cell.y 520315400.984
QSize100
cell.d 4885229.57737cell.maX 5127326.87466cell.x 137318020.609cell.y 518603514.578
QSize99
cell.d 5092255.18371cell.maX 5122517.34587cell.x 141319555.084cell.y 520636379.686
QSize98
cell.d 3184210.16745cell.maX 5120988.54582cell.x 127902645.375cell.y 528703644.375
QSize97
cell.d 4877291.97501cell.maX 5119389.27231cell.x 136975643.328cell.y 518603514.578
QSize96
cell.d 4875858.15789cell.maX 5117955.45519cell.x 141426547.984cell.y 520315400.984
QSize95
cell.d 5048851.06825cell.maX 5109375.39258cell.x 141383750.824cell.y 520614981.105
QSi

In [44]:
polygon_families

[[[(147658159, 516106808),
   (147559339, 516455378),
   (147378099, 516873706),
   (146613004, 517374483),
   (146537610, 517782580),
   (146023098, 518078268),
   (146037449, 518476488),
   (146529376, 518791843),
   (147049071, 519227010),
   (147048320, 519767833),
   (147341510, 520192472),
   (147527410, 520583200),
   (147265996, 520925819),
   (146856436, 521496062),
   (147146027, 522368910),
   (145834450, 523016589),
   (145538141, 523496975),
   (145442241, 524263519),
   (146283757, 524855142),
   (146207758, 525583599),
   (145757245, 526189857),
   (144148384, 527005259),
   (143256775, 527608542),
   (141461628, 528243281),
   (141585995, 528803489),
   (141758085, 529754736),
   (143389984, 530481673),
   (143699270, 531595046),
   (144078177, 532194176),
   (144439734, 532745634),
   (143807188, 533212440),
   (143260023, 533116626),
   (143013907, 532888490),
   (142501743, 532573553),
   (142021654, 532601458),
   (141573747, 532674660),
   (141046785, 532769056),
 